In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets.samples_generator import make_regression 
%matplotlib inline

sizes = np.loadtxt('flat.areas').reshape((26,1)) #załaduj wektor rozmiarów mieszkań
prices = np.loadtxt('flat.prices').reshape((26,1)) #załaduj wektor cen mieszkań
samples_cnt = len(prices) #pobierz liczność par rozmiar->cena
weight0 = 0.0 # inicjalizuj wagi
weight1 = 0.0 #
learning_rate = 0.0001 #stala uczenia
maxIteration = 100000 #liczba iteracji

for i in range(maxIteration):
    weight0 = weight0
    weight1 = weight1
    
    #Zadanie1: 
    predicted_prices = []
    #uzupełnij listę predicted_prices tak, aby dla każdego metrazu budynku z listy 'sizes' wyznaczyć cenę tegoż
    #budynku przy użyciu aktualnych wag modelu liniowego
    
    #Zadanie2:
    #napisz kod, który iteracyjnie poprawiać będzie wagi tak, aby ostatecznie wyznaczyły prostą,
    #która najlepiej odwzoru je zależność metraż -> cena
    
plt.plot(sizes, prices, "x")
if len(predicted_prices) > 0:
    plt.plot(sizes, predicted_prices, "r-")
plt.title('Ceny mieszkan w zaleznosci od metrazu')
plt.xlabel('metraz (m^2)')
plt.ylabel('Cena mieskania (tys. zł)')
plt.show()
print("Wyznaczone wartości wag -> w0:", weight0, "w1:", weight1)

In [ ]:
from helpers import visualize_cost_function
import numpy as np

def loss_function(sizes, prices, weight0, weight1):
    predictions = [weight0 + weight1*sizes[i] for i in range(len(prices))]
    return sum([(predictions[i] - prices[i])**2 for i in range(len(prices))])/(2*len(prices))


def loss_function2(sizes, prices, weight0, weight1):
    predictions = [weight0 + weight1*sizes[i] for i in range(len(prices))]
    return sum([(np.sin(predictions[i]/prices[i]**2) * (predictions[i]*prices[i]**3)) for i in range(len(prices))])/(2*len(prices))


#Poniższy kod wyznacza wartości funkcji kosztu w dziedzinie wartości atrybutów w0 i w1
w0_values = np.arange(-10, 10, 0.5)
w1_values = np.arange(-10, 10, 0.5)

sizes_transformed = (sizes - np.mean(sizes)) / np.std(sizes)
prices_transformed = (prices - np.mean(prices)) / np.std(prices)

w0_values = np.arange(-100, 100, 5)
w1_values = np.arange(-100, 100, 5)

visualize_cost_function(sizes_transformed, prices_transformed, w0_values, w1_values, loss_function)
visualize_cost_function(sizes_transformed, prices_transformed, w0_values, w1_values, loss_function2)

#Zad 3
#Po wykonaniu tego fragmentu kodu zobaczysz wizualizacje dwóch różnych funkcji kosztu. 
#Pierwsza z nich obrazuje błąd średniokwadratowy, druga - jakąś arbitralnie wybraną funkcję
#Zastanów się, dlaczego używa się często tej pierwszej w procesie uczenia spadkiem gradientowym, a nie tej drugiej?
#Co sprawia, że funkcja kosztu jest dobra?


In [ ]:
#regresja wielomianowa

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split # model_selection w starszych wersjach -> cross_validation
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

#funkcja, ktora chcemy aproksymowac
def f(x): 
	return -(3*x)+(7*x**2)-(3*x**3)

x = np.linspace(0,1,100) #Stworzenie przestrzeni w której będziemy wizualizować aproksymowaną funkcję (x od 0 do 1 z krokiem 1/100)

X = np.random.uniform(0, 1, size=50)[:, np.newaxis] #Wylosowanie przykladow (os X)
y = f(X) + np.random.normal(scale=0.3, size=50)[:, np.newaxis] #Wygenerowanie punktów zgodnie z funkcją f(x) z uwzględnieniem szumu

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7) #Podzial danych na zbior treningowy i testowy


#Pomocnicza funkcja do wizualizacji
def plot_approximation(est, ax, label=None):
    ax.plot(x, f(x), color='green')
    ax.scatter(X_train, y_train, s=10) #Dodanie punktow zbioru treningowego do wizualizacji
    ax.scatter(X_test, y_test, s=10, color='green', alpha=0.2) #Dodanie punktow zbioru testowego do wizualizacji
    ax.plot(x, est.predict(x[:, np.newaxis]), color='red', label=label) #Wizualizacja wygenerowanej funkcji
    ax.set_ylim((-2, 2))
    ax.set_xlim((0, 1))
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.legend(loc='upper right')

#Uzycie sklearn do wygenerowania regresji wielomianowej stopnia 0,1,3,9
fig, axes = plt.subplots(2, 2, figsize=(8, 5))
for ax, degree in zip(axes.ravel(), [0, 1, 3, 9]):
    est = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    est.fit(X_train, y_train)
    plot_approximation(est, ax, label='stopien=%d' % degree)
plt.tight_layout()
plt.show()


#Uzycie sklearn do wygenerowania regresji wielomianowej stopnia 0..15, w celu weryfikacji bledu na zbiorach uczacym i testowym
train_error = np.empty(15)
test_error = np.empty(15)
for degree in range(15):
    est = make_pipeline(PolynomialFeatures(degree), LinearRegression()) #Zdefiniowanie typu modelu
    est.fit(X_train, y_train)  #Uczenie parametrów
    train_error[degree] = mean_squared_error(y_train, est.predict(X_train)) #Zapisanie bledu zb. uczacego dla zadanego stopnia wielomianu
    test_error[degree] = mean_squared_error(y_test, est.predict(X_test)) #Zapisanie bledu zb. testowego dla zadanego stopnia wielomianu
    
    
#Wizualizacja bledu. Odpowiednio na zbiorze uczacym i testowym
plt.plot(np.arange(15), train_error, color='green', label='blad na zbiorze treningowym')
plt.plot(np.arange(15), test_error, color='red', label='blad na zbiorze testowym')
plt.legend(loc = 'upper right')
plt.ylim((0.0, 1e0))
plt.ylabel('MSE')
plt.xlabel('stopien wielomianu')
plt.legend(loc='lower left')
plt.show()



#Zadanie 4:
#Zastanow sie dlaczego powyzej pewnego stopnia wielomianu blad na zbiorze testowym staje sie coraz wiekszy?
#Po co wprowadza się zbiór testowy?


#Zadanie 5:
#Podmieniając LinearRegression przez Ridge, sprawdź jaki wpływ ma regularyzacja na błąd testowy. 
#Zaobserwuj różnicę w wyglądzie wygenerowanej krzywej.